In [2]:
from sklearn import *
import sklearn
import pandas as pd
import numpy as np
import xgboost as xgb

train = pd.read_csv('training_variants.csv')
test = pd.read_csv('test_variants.csv')
trainx = pd.read_csv('training_text.csv', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])
testx = pd.read_csv('test_text.csv', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])

train = pd.merge(train, trainx, how='left', on='ID').fillna('')
y = train['Class'].values
train = train.drop(['Class'], axis=1)

test = pd.merge(test, testx, how='left', on='ID').fillna('')
pid = test['ID'].values
df_all = pd.concat((train, test), axis=0, ignore_index=True)

In [3]:
#if gene name appears in Text, 1: appeared, 0: not
df_all['Gene_Share'] = df_all.apply(lambda r: sum([1 for w in r['Gene'].split(' ') if w in r['Text'].split(' ')]), axis=1)

,ID,Gene,Variation,Text,Gene_Share
0,0,FAM58A,Truncating Mutations,Cyclin-dependent kinases (CDKs) regulate a var...,1
1,1,CBL,W802*,Abstract Background Non-small cell lung canc...,1
2,2,CBL,Q249E,Abstract Background Non-small cell lung canc...,1
3,3,CBL,N454D,Recent evidence has demonstrated that acquired...,1
4,4,CBL,L399V,Oncogenic mutations in the monomeric Casitas B...,1


In [7]:
#if variation appears in Text
df_all['Variation_Share'] = df_all.apply(lambda r: sum([1 for w in r['Variation'].split(' ') if w in r['Text'].split(' ')]), axis=1)

In [11]:
for i in range(56):
    df_all['Gene_'+str(i)] = df_all['Gene'].map(lambda x: str(x[i]) if len(x)>i else '')
    df_all['Variation'+str(i)] = df_all['Variation'].map(lambda x: str(x[i]) if len(x)>i else '')

In [12]:
df_all.head()

,ID,Gene,Variation,Text,Gene_Share,Variation_Share,Gene_0,Variation0,Gene_1,Variation1,...,Gene_51,Variation51,Gene_52,Variation52,Gene_53,Variation53,Gene_54,Variation54,Gene_55,Variation55
0,0,FAM58A,Truncating Mutations,Cyclin-dependent kinases (CDKs) regulate a var...,1,1,F,T,A,r,...,,,,,,,,,,
1,1,CBL,W802*,Abstract Background Non-small cell lung canc...,1,1,C,W,B,8,...,,,,,,,,,,
2,2,CBL,Q249E,Abstract Background Non-small cell lung canc...,1,1,C,Q,B,2,...,,,,,,,,,,
3,3,CBL,N454D,Recent evidence has demonstrated that acquired...,1,1,C,N,B,4,...,,,,,,,,,,
4,4,CBL,L399V,Oncogenic mutations in the monomeric Casitas B...,1,1,C,L,B,3,...,,,,,,,,,,


In [13]:
gen_var_lst = sorted(list(train.Gene.unique()) + list(train.Variation.unique()))


In [16]:
gen_var_lst = [x for x in gen_var_lst if len(x.split(' '))==1]


In [17]:
gen_var_lst

['1_2009trunc',
 '2010_2471trunc',
 '256_286trunc',
 '385_418del',
 '422_605trunc',
 '533_534del',
 '534_536del',
 '550_592del',
 '560_561insER',
 '596_619splice',
 '963_D1010splice',
 '981_1028splice',
 'A1020V',
 'A1022E',
 'A1065T',
 'A1066V',
 'A1099T',
 'A111P',
 'A1131T',
 'A113_splice',
 'A1170V',
 'A11_G12insGA',
 'A1200V',
 'A120S',
 'A121E',
 'A121P',
 'A121V',
 'A122*',
 'A1234T',
 'A126D',
 'A126G',
 'A126S',
 'A126V',
 'A134D',
 'A1374V',
 'A1459P',
 'A146T',
 'A146V',
 'A148T',
 'A149P',
 'A1519T',
 'A151T',
 'A159T',
 'A161S',
 'A161T',
 'A1669S',
 'A1685S',
 'A1701P',
 'A1708E',
 'A1708V',
 'A171V',
 'A1752P',
 'A1752V',
 'A1789S',
 'A1789T',
 'A1823T',
 'A1830T',
 'A1843P',
 'A1843T',
 'A18D',
 'A197T',
 'A19V',
 'A2034V',
 'A205T',
 'A209T',
 'A211D',
 'A232V',
 'A2351G',
 'A23E',
 'A2425T',
 'A246P',
 'A263V',
 'A2643G',
 'A2717S',
 'A272V',
 'A2770T',
 'A290T',
 'A298T',
 'A339V',
 'A347T',
 'A349P',
 'A34D',
 'A36P',
 'A389T',
 'A391E',
 'A39P',
 'A40E',
 'A41P',
 

In [18]:
i_ = 0
#commented for Kaggle Limits
for gen_var_lst_itm in gen_var_lst:
    if i_ % 100 == 0: 
        print(i_)
    df_all['GV_'+str(gen_var_lst_itm)] = df_all['Text'].map(lambda x: str(x).count(str(gen_var_lst_itm)))
    i_ += 1

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000


In [16]:
df_all['Gene_Share'] = df_all.apply(lambda r: sum([1 for w in r['Gene'].split(' ') if w in r['Text'].split(' ')]), axis=1)
df_all['Variation_Share'] = df_all.apply(lambda r: sum([1 for w in r['Variation'].split(' ') if w in r['Text'].split(' ')]), axis=1)

#commented for Kaggle Limits
for i in range(56):
    df_all['Gene_'+str(i)] = df_all['Gene'].map(lambda x: str(x[i]) if len(x)>i else '')
    df_all['Variation'+str(i)] = df_all['Variation'].map(lambda x: str(x[i]) if len(x)>i else '')


gen_var_lst = sorted(list(train.Gene.unique()) + list(train.Variation.unique()))
print(len(gen_var_lst))
gen_var_lst = [x for x in gen_var_lst if len(x.split(' '))==1]
print(len(gen_var_lst))
i_ = 0
#commented for Kaggle Limits
for gen_var_lst_itm in gen_var_lst:
    if i_ % 100 == 0: 
        print(i_)
    df_all['GV_'+str(gen_var_lst_itm)] = df_all['Text'].map(lambda x: str(x).count(str(gen_var_lst_itm)))
    i_ += 1

for c in df_all.columns:
    if df_all[c].dtype == 'object':
        if c in ['Gene','Variation']:
            lbl = preprocessing.LabelEncoder()
            df_all[c+'_lbl_enc'] = lbl.fit_transform(df_all[c].values)  
            df_all[c+'_len'] = df_all[c].map(lambda x: len(str(x)))
            df_all[c+'_words'] = df_all[c].map(lambda x: len(str(x).split(' ')))
        elif c != 'Text':
            lbl = preprocessing.LabelEncoder()
            df_all[c] = lbl.fit_transform(df_all[c].values)
        if c=='Text': 
            df_all[c+'_len'] = df_all[c].map(lambda x: len(str(x)))
            df_all[c+'_words'] = df_all[c].map(lambda x: len(str(x).split(' '))) 

train = df_all.iloc[:len(train)]
test = df_all.iloc[len(train):]

class cust_regression_vals(sklearn.base.BaseEstimator, sklearn.base.TransformerMixin):
    def fit(self, x, y=None):
        return self
    def transform(self, x):
        x = x.drop(['Gene', 'Variation','ID','Text'],axis=1).values
        return x

class cust_txt_col(sklearn.base.BaseEstimator, sklearn.base.TransformerMixin):
    def __init__(self, key):
        self.key = key
    def fit(self, x, y=None):
        return self
    def transform(self, x):
        return x[self.key].apply(str)


3260
3091
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000


In [17]:
print('Pipeline...')
fp = pipeline.Pipeline([
    ('union', pipeline.FeatureUnion(
        n_jobs = -1,
        transformer_list = [
            ('standard', cust_regression_vals()),
            ('pi1', pipeline.Pipeline([('Gene', cust_txt_col('Gene')), ('count_Gene', feature_extraction.text.CountVectorizer(analyzer=u'char', ngram_range=(1, 8))), ('tsvd1', decomposition.TruncatedSVD(n_components=20, n_iter=25, random_state=12))])),
            ('pi2', pipeline.Pipeline([('Variation', cust_txt_col('Variation')), ('count_Variation', feature_extraction.text.CountVectorizer(analyzer=u'char', ngram_range=(1, 8))), ('tsvd2', decomposition.TruncatedSVD(n_components=20, n_iter=25, random_state=12))])),
            #commented for Kaggle Limits
            ('pi3', pipeline.Pipeline([('Text', cust_txt_col('Text')), ('tfidf_Text', feature_extraction.text.TfidfVectorizer(ngram_range=(1, 2))), ('tsvd3', decomposition.TruncatedSVD(n_components=50, n_iter=25, random_state=12))]))
        ])
    )])

train = fp.fit_transform(train); print(train.shape)
test = fp.transform(test); print(test.shape)

y = y - 1 #fix for zero bound array



Pipeline...
(3321, 3303)
(5668, 3303)


In [18]:
denom = 0
fold = 20 #Change to 5, 1 for Kaggle Limits
for i in range(fold):
    params = {
        'eta': 0.03333,
        'max_depth': 4,
        'objective': 'multi:softprob',
        'eval_metric': 'mlogloss',
        'num_class': 9,
        'seed': i,
        'silent': True
    }
    x1, x2, y1, y2 = model_selection.train_test_split(train, y, test_size=0.18, random_state=i)
    watchlist = [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]
    model = xgb.train(params, xgb.DMatrix(x1, y1), 1000,  watchlist, verbose_eval=50, early_stopping_rounds=100)
    score1 = metrics.log_loss(y2, model.predict(xgb.DMatrix(x2), ntree_limit=model.best_ntree_limit), labels = list(range(9)))
    print(score1)
    #if score < 0.9:
    if denom != 0:
        pred = model.predict(xgb.DMatrix(test), ntree_limit=model.best_ntree_limit+80)
        preds += pred
    else:
        pred = model.predict(xgb.DMatrix(test), ntree_limit=model.best_ntree_limit+80)
        preds = pred.copy()
    denom += 1
    submission = pd.DataFrame(pred, columns=['class'+str(c+1) for c in range(9)])
    submission['ID'] = pid
    submission.to_csv('submission_xgb_fold_'  + str(i) + '.csv', index=False)
preds /= denom
submission = pd.DataFrame(preds, columns=['class'+str(c+1) for c in range(9)])
submission['ID'] = pid
submission.to_csv('submission_xgb.csv', index=False)

[0]	train-mlogloss:2.1401	valid-mlogloss:2.14783
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 100 rounds.
[50]	train-mlogloss:1.09983	valid-mlogloss:1.28506
[100]	train-mlogloss:0.815733	valid-mlogloss:1.09792
[150]	train-mlogloss:0.667969	valid-mlogloss:1.025
[200]	train-mlogloss:0.569891	valid-mlogloss:0.989683
[250]	train-mlogloss:0.495857	valid-mlogloss:0.972529
[300]	train-mlogloss:0.436233	valid-mlogloss:0.961683
[350]	train-mlogloss:0.388347	valid-mlogloss:0.954649
[400]	train-mlogloss:0.345309	valid-mlogloss:0.951315
[450]	train-mlogloss:0.308374	valid-mlogloss:0.948311
[500]	train-mlogloss:0.275986	valid-mlogloss:0.947559
[550]	train-mlogloss:0.247036	valid-mlogloss:0.949231
Stopping. Best iteration:
[467]	train-mlogloss:0.296925	valid-mlogloss:0.947183

0.947182750212
[0]	train-mlogloss:2.14187	valid-mlogloss:2.14928
Multiple eval metrics have been passed: 'valid-mlogloss' will be